# Updating the imported Fishing Vessels data

In [ ]:
# import modules needed and database connection from setup file

# %run 02_setup.py

The setup file is disabled by default as this notebook is called in the project diary which already implements the set ups. It can be enabled here if this notebook needs to be ran independently.

## Store the data in a MongoDB database

The data files for UK registered fishing vessels are found in the folder:
```
    Analysis_data/FishingVessels
```

In [8]:
# check and remove existing fishing vessels data
ves_land_db.drop_collection('uk_fishing_collection')
ves_land_db.drop_collection('uk_adminport_geoloc')

{'ok': 0.0,
 'errmsg': 'ns not found',
 'code': 26,
 'codeName': 'NamespaceNotFound'}

In [9]:
# create a uk fishing collection
ves_land_db.drop_collection('uk_fishing_collection')
uf_collection = ves_land_db['uk_fishing_collection']

In [10]:
# get the names of the files in the directory
os.listdir("Analysis_data/FishingVessels")

['January_2017_Over_10m_vessel_list.xls',
 'January_2017_Under_10m_vessel_list.xls',
 'January_2018_Over_10m_vessel_list.xls',
 'January_2018_Under_10m_vessel_list.xls',
 'January_2021_Over_10m_vessel_list.xls',
 'January_2021_Under_10m_vessel_list.xls',
 'Jan_2019_Over_10m_vessel_list.xls',
 'Jan_2019_Under_10m_vessel_list.xls',
 'Jan_2020_Over_10m_vessel_list.xls',
 'Jan_2020_Under_10m_vessel_list.xls']

Before importing the files into mongoDB they were analysed and cleaned.

The files were imported in order based on Length. This is because it is assumed that since they capture similar data, they are likely to have the same column. This may require less amendments. They can then be merged into a single file based on length. Two final two data frames can be matched by column names then added to mongodb.

As the excel files did not state the years, a 'For year' column is added to each file, with the corresponding year, when imported.

When adding files to the dataframe, it is acknowledged that the report are done in Jan but represents the following year. For example the 2018 data is for data captured in the 2017. To accurately represent this, the a column "For year" is added with the year of the report minus 1. So 2017 - 1 means the report is for the year 2016.

#### Importing data with years for vessels of over 10 metres

The sheet was reviewed before import so it was known the the data existed on the first sheet of each document and the line the data started on was 5.

In [11]:
# initialise an empty df to input all entries
uk_over10_df = pd.DataFrame()

# get all data from the directory for vessels over 10m
for i in glob.glob("Analysis_data/FishingVessels/*Over_10m_vessel_list.xls"):
    temp_df = pd.read_excel(i,
                        sheet_name=0,
                        skiprows=4
                        )

    # get the year from the string and minus 1 
    # add to column that shows the reporting year
    temp_df["For Year"] = (int(re.search(r"_(\d{4})_",i).group(1)) - 1)

    # add the data to the main df
    uk_over10_df =  uk_over10_df.append(temp_df, ignore_index=1)
    
# preview first and last 3 entries
uk_over10_df.head(3).append(uk_over10_df.tail(3))

,Administrative port,Home port,Port letters and numbers,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine power,Vessel capacity units,Year built,Hull,Country of construction,Shellfish licence (Y/N),Scallop licence (Y/N),For Year,Licence Category
0,ABERDEEN,ABERDEEN,WK46,LILY V,A10911,44163,NON-SECTOR,10.30,10.36,82.0,82.632,1983.0,W,GBR,Y,N,2016,NaN
1,ABERDEEN,ABERDEEN,AH4,BONNIE LASS II,C20280,11955,NON-SECTOR,10.40,9.41,140.0,103.872,2012.0,F,GBR,Y,N,2016,NaN
2,ABERDEEN,ABERDEEN,A441,BOY GORDON V,B10713,40414,ABERDEEN FPO LTD,11.28,28.06,94.0,94.639,1989.0,F,GBR,Y,N,2016,NaN
6267,ULLAPOOL,ULLAPOOL,UL1,ONWARD,B10067,40995,SCOTTISH FISHERMENS ORGANISATION,11.62,16.74,149.2,119.198,1988.0,F,GBR,Y,Y,2019,OVER 10 METRE
6268,ULLAPOOL,ULLAPOOL,UL584,LADY NICOLA,C19830,40454,NON-SECTOR,11.39,15.33,164.0,121.410,1989.0,F,GBR,N,N,2019,OVER 10 METRE
6269,ULLAPOOL,ULLAPOOL,UL595,COURAGE,C20423,41687,NON-SECTOR,12.90,18.86,147.0,120.330,1980.0,S,SWE,N,N,2019,OVER 10 METRE


#### Cleaning the data for over 10m - THIS SECTION IS AN UPDATE

In [12]:
# check the df details
uk_over10_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6270 entries, 0 to 6269
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Administrative port                     6270 non-null   object 
 1   Home port                               6270 non-null   object 
 2   Port letters and numbers                6270 non-null   object 
 3   Vessel name                             6270 non-null   object 
 4   Registry of Shipping and Seamen number  6270 non-null   object 
 5   Licence number                          6270 non-null   int64  
 6   Fish producer organisation              6270 non-null   object 
 7   Overall length                          6270 non-null   float64
 8   Registered tonnage                      6270 non-null   float64
 9   Engine power                            6270 non-null   float64
 10  Vessel capacity units                   6270 non-null   floa

In [13]:
#Check the column types
uk_over10_df.dtypes

Administrative port                        object
Home port                                  object
Port letters and numbers                   object
Vessel name                                object
Registry of Shipping and Seamen number     object
Licence number                              int64
Fish producer organisation                 object
Overall length                            float64
Registered tonnage                        float64
Engine power                              float64
Vessel capacity units                     float64
Year built                                float64
Hull                                       object
Country of construction                    object
Shellfish  licence (Y/N)                   object
Scallop licence (Y/N)                      object
For Year                                    int64
Licence Category                           object
dtype: object

All types seem to match what they represent

In [14]:
# check for null values
uk_over10_df.isna().sum()

Administrative port                          0
Home port                                    0
Port letters and numbers                     0
Vessel name                                  0
Registry of Shipping and Seamen number       0
Licence number                               0
Fish producer organisation                   0
Overall length                               0
Registered tonnage                           0
Engine power                                 0
Vessel capacity units                        0
Year built                                  82
Hull                                        28
Country of construction                    278
Shellfish  licence (Y/N)                     1
Scallop licence (Y/N)                       23
For Year                                     0
Licence Category                          1236
dtype: int64

The datasets are unable to state what year some vessels where built, however the vessels can still be used for analysis. The unknown years can temporarily be represented by 0 as the column needs to be a number. 

In [15]:
# replace null values with 0
uk_over10_df["Year built"].fillna(0, inplace=True)

Based on the notes tab of the vessels list, Unknown values are represented with "U". Therefore this can be used to replaced the null values since it is not possible to know what material the vessel has, via the documents.

In [16]:
uk_over10_df["Hull"].fillna("U", inplace=True)

Based on the notes tab of the vessels list, Unknown values are represented with "UNK". Therefore this can be used to replaced the null values since it is not possible to know what material the vessel has, via the documents.

In [17]:
# replace null Country of construction to UNK
uk_over10_df["Country of construction"].fillna("UNK", inplace=True)

Since there is just one entry, it is possible that the null values got updated in later years. It can then be updated to match.

In [18]:
# check which item in the Shellfish  licence (Y/N) column is null
uk_over10_df[pd.isnull(uk_over10_df["Shellfish  licence (Y/N)"])].head()

,Administrative port,Home port,Port letters and numbers,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine power,Vessel capacity units,Year built,Hull,Country of construction,Shellfish licence (Y/N),Scallop licence (Y/N),For Year,Licence Category
587,KIRKWALL,KIRKWALL,K193,RUSSA TAIGN,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,NaN,NaN,2016,NaN


In [19]:
# check other entries for same licence number
uk_over10_df[uk_over10_df["Licence number"]==41679]

,Administrative port,Home port,Port letters and numbers,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine power,Vessel capacity units,Year built,Hull,Country of construction,Shellfish licence (Y/N),Scallop licence (Y/N),For Year,Licence Category
587,KIRKWALL,KIRKWALL,K193,RUSSA TAIGN,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,NaN,NaN,2016,NaN
1857,KIRKWALL,KIRKWALL,K193,RUSSA TAIGN,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,N,N,2017,OVER 10 METRE
3039,KIRKWALL,KIRKWALL,K148,CASPIAN K,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,N,N,2020,OVER 10 METRE
4733,KIRKWALL,KIRKWALL,K193,RUSSA TAIGN,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,N,N,2018,OVER 10 METRE
5393,KIRKWALL,KIRKWALL,K193,RUSSA TAIGN,C16193,41679,SCOTTISH FISHERMENS ORGANISATION,27.41,354.0,663.27,532.005,1997.0,S,GBR,N,N,2019,OVER 10 METRE


In [20]:
# replace entry to "N" to match similar entries
uk_over10_df["Shellfish  licence (Y/N)"].fillna("N", inplace=True)

It is assumed that no Scallop licence entry meant that the vessel was not issued one for that time. So the null entries where replaced to "N" 

In [21]:
# replace Scallop licence (Y/N) entries to "N" 
uk_over10_df["Scallop licence (Y/N)"].fillna("N", inplace=True)

The report for 2016 does not contain columns for "Licence Category", this can be represented as unknown for that year even thought they may have been updated in the next year.

In [22]:
# replace Licence cat to unknown
uk_over10_df["Licence Category"].fillna("Unknown", inplace=True)

In [23]:
# check for null values
uk_over10_df.isna().sum()

Administrative port                       0
Home port                                 0
Port letters and numbers                  0
Vessel name                               0
Registry of Shipping and Seamen number    0
Licence number                            0
Fish producer organisation                0
Overall length                            0
Registered tonnage                        0
Engine power                              0
Vessel capacity units                     0
Year built                                0
Hull                                      0
Country of construction                   0
Shellfish  licence (Y/N)                  0
Scallop licence (Y/N)                     0
For Year                                  0
Licence Category                          0
dtype: int64

All adjustment ensure that all Null values are handled so no missing values are imported into the mongoDB database

#### Importing data with years for vessels of under 10 metres

In [24]:
# initialise an empty df to input all entries
uk_under10_df = pd.DataFrame()

# get all data from the directory for vessels over 10m
for i in glob.glob("Analysis_data/FishingVessels/*Under_10m_vessel_list.xls"):
    temp_df = pd.read_excel(i,
                        sheet_name=0,
                        skiprows=4
                        )

    # get the year from the string and minus 1 
    # add a column that shows the reporting year
    temp_df["For Year"] = int(re.search(r"_(\d{4})_",i).group(1)) - 1

    # add the data to the main df
    uk_under10_df =  uk_under10_df.append(temp_df, ignore_index=1)
    
# preview first and last 3 entries
uk_under10_df.head(3).append(uk_under10_df.tail(3))

,Administrative port,Home port,Port letters and number,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine Power (kw),Vessel Capacity Units,Year Built,Hull Material,Country of Build,Shellfish Licence licence (Y/N),Scallop Licence (Y/N),For Year,Licence Category
0,ABERDEEN,ABERDEEN,K4,CORNUCOPIA,C18403,20166,NON-SECTOR,4.69,0.78,14.00,14.179,1970.0,F,GBR,Y,N,2016,NaN
1,ABERDEEN,ABERDEEN,A974,RACHEL,C20472,25923,NON-SECTOR,4.77,0.67,5.96,11.984,2004.0,F,GBR,Y,N,2016,NaN
2,ABERDEEN,ABERDEEN,A946,SHELLFISHER,C19371,30508,NON-SECTOR,4.77,0.58,15.00,16.147,2000.0,F,GBS,Y,N,2016,NaN
21071,ULLAPOOL,UNKNOWN,SY12,IONA LOUISE,C17341,33855,NON-SECTOR,5.82,1.64,20.00,23.783,2001.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)
21072,ULLAPOOL,UNKNOWN,SY2,NORTHERN LIGHTS,A24446,22265,NON-SECTOR,9.50,8.12,69.00,64.300,1978.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)
21073,ULLAPOOL,UNKNOWN,UL597,SOPHIE JANE,C17525,22724,NON-SECTOR,5.62,1.03,37.30,27.575,1989.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)


#### Cleaning the data for under 10m - THIS SECTION IS AN UPDATE

In [25]:
#Check the column types
uk_under10_df.dtypes

Administrative port                        object
Home port                                  object
Port letters and number                    object
Vessel name                                object
Registry of Shipping and Seamen number     object
Licence number                              int64
Fish producer organisation                 object
Overall length                            float64
Registered tonnage                        float64
Engine Power (kw)                         float64
Vessel Capacity Units                     float64
Year Built                                float64
Hull Material                              object
Country of Build                           object
Shellfish Licence  licence (Y/N)           object
Scallop Licence (Y/N)                      object
For Year                                    int64
Licence Category                           object
dtype: object

All types seem to match what they represent

In [26]:
# check for null values
uk_under10_df.isna().sum()

Administrative port                          0
Home port                                    0
Port letters and number                      0
Vessel name                                  0
Registry of Shipping and Seamen number       0
Licence number                               0
Fish producer organisation                   0
Overall length                               0
Registered tonnage                           0
Engine Power (kw)                            0
Vessel Capacity Units                        0
Year Built                                1450
Hull Material                              159
Country of Build                          1978
Shellfish Licence  licence (Y/N)           161
Scallop Licence (Y/N)                       25
For Year                                     0
Licence Category                          4248
dtype: int64

The datasets are unable to state what year some vessels where built, however the vessels can still be used for analysis. The unknown years can temporarily be represented by 0 as the column needs to be a number. 

In [27]:
# replace null values with 0
uk_under10_df["Year Built"].fillna(0, inplace=True)

Based on the notes tab of the vessels list, Unknown values are represented with "U". Therefore this can be used to replaced the null values since it is not possible to know what material the vessel has, via the documents.

In [28]:
uk_under10_df["Hull Material"].fillna("U", inplace=True)

Based on the notes tab of the vessels list, Unknown values are represented with "UNK". Therefore this can be used to replaced the null values since it is not possible to know what material the vessel has, via the documents.

In [29]:
# replace null Country of construction to UNK
uk_under10_df["Country of Build"].fillna("UNK", inplace=True)

Since there is just one entry, it is possible that the null values got updated in later years. It can then be updated to match.

In [30]:
# replace entry to "N" to match similar entries
uk_under10_df["Shellfish Licence  licence (Y/N)"].fillna("N", inplace=True)

It is assumed that no Scallop licence entry meant that the vessel was not issued one for that time. So the null entries where replaced to "N" 

In [31]:
# replace Scallop licence (Y/N) entries to "N" 
uk_under10_df["Scallop Licence (Y/N)"].fillna("N", inplace=True)

The report for 2016 does not contain columns for "Licence Category", this can be represented as unknown for that year even thought they may have been updated in the next year.

In [32]:
# replace Licence cat to unknown
uk_under10_df["Licence Category"].fillna("Unknown", inplace=True)

In [33]:
# check for null values
uk_over10_df.isna().sum()

Administrative port                       0
Home port                                 0
Port letters and numbers                  0
Vessel name                               0
Registry of Shipping and Seamen number    0
Licence number                            0
Fish producer organisation                0
Overall length                            0
Registered tonnage                        0
Engine power                              0
Vessel capacity units                     0
Year built                                0
Hull                                      0
Country of construction                   0
Shellfish  licence (Y/N)                  0
Scallop licence (Y/N)                     0
For Year                                  0
Licence Category                          0
dtype: int64

All adjustment ensure that all Null values are handled so no missing values are imported into the mongoDB database

#### Comparing both dataframes

In [34]:
# check for a difference in column names of the two datasets
uk_over10_df.columns.difference(uk_under10_df.columns)

Index(['Country of construction', 'Engine power', 'Hull',
       'Port letters and numbers', 'Scallop licence (Y/N)',
       'Shellfish  licence (Y/N)', 'Vessel capacity units', 'Year built'],
      dtype='object')

After investigating the names of the columns of both data it is deduced that:
- Country of construction and Country of Build represent the same data so Country of Build was renamed to Country of construction
- Engine power and Engine Power (kw) is the same so Engine Power (kw) was renamed to Engine power
- Hull and Hull Material is the same so Hull Material was renamed to Hull
- 'Port letters and numbers' and 'Port letters and number' is the same so Port letters and number was renamed to Port letters and numbers
- Scallop licence (Y/N) and Scallop Licence (Y/N) is the same so Scallop Licence (Y/N) was renamed to Scallop licence (Y/N)
- Shellfish  licence (Y/N) and Shellfish Licence  licence (Y/N) is the same so Shellfish Licence  licence (Y/N) was renamed to Shellfish licence (Y/N)
- Vessel capacity units and Vessel Capacity Units is the same so Vessel Capacity Units was replaced to Vessel capacity units
- Year built and Year Built is the same so Year Built was renamed to Year built

all renaming were done in uk_under10_df

In [35]:
uk_under10_df.rename(columns={'Country of Build': 'Country of construction',
                              'Engine Power (kw)': 'Engine power',
                              'Hull Material': 'Hull',
                              'Port letters and number': 'Port letters and numbers',
                              'Scallop Licence (Y/N)': 'Scallop licence (Y/N)',
                              'Shellfish Licence  licence (Y/N)': 'Shellfish  licence (Y/N)',
                              'Vessel Capacity Units': 'Vessel capacity units',
                              'Year Built': 'Year built'}, inplace=True)


In [36]:
# re-check for a difference in column names of the two datasets
uk_over10_df.columns.difference(uk_under10_df.columns)

Index([], dtype='object')

As the result returns empty, the columns names on both datasets now match.

#### Inserting the data to mongoDB

Now that both Over 10m and Under 10m contain data with similar headings they can be entered into mongoDB as a single collection

In [37]:
# insert uk_over10_df into mongoDB uf_collection
uf_collection.insert_many(uk_over10_df.to_dict(orient='records'))

# insert uk_under10_df into mongoDB uf_collection
uf_collection.insert_many(uk_under10_df.to_dict(orient='records'))

In [38]:
# preview first 3 documents where uk_over10_df was inserted
pd.DataFrame(uf_collection.find({})).head(3)

,_id,Administrative port,Home port,Port letters and numbers,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine power,Vessel capacity units,Year built,Hull,Country of construction,Shellfish licence (Y/N),Scallop licence (Y/N),For Year,Licence Category
0,629e7390924ea1d650dede0b,ABERDEEN,ABERDEEN,WK46,LILY V,A10911,44163,NON-SECTOR,10.30,10.36,82.0,82.632,1983.0,W,GBR,Y,N,2016,Unknown
1,629e7390924ea1d650dede0c,ABERDEEN,ABERDEEN,AH4,BONNIE LASS II,C20280,11955,NON-SECTOR,10.40,9.41,140.0,103.872,2012.0,F,GBR,Y,N,2016,Unknown
2,629e7390924ea1d650dede0d,ABERDEEN,ABERDEEN,A441,BOY GORDON V,B10713,40414,ABERDEEN FPO LTD,11.28,28.06,94.0,94.639,1989.0,F,GBR,Y,N,2016,Unknown


In [39]:
# preview last 3 documents where uk_under10_df was inserted
pd.DataFrame(uf_collection.find({})).tail(3)

,_id,Administrative port,Home port,Port letters and numbers,Vessel name,Registry of Shipping and Seamen number,Licence number,Fish producer organisation,Overall length,Registered tonnage,Engine power,Vessel capacity units,Year built,Hull,Country of construction,Shellfish licence (Y/N),Scallop licence (Y/N),For Year,Licence Category
27341,629e7392924ea1d650df48d8,ULLAPOOL,UNKNOWN,SY12,IONA LOUISE,C17341,33855,NON-SECTOR,5.82,1.64,20.0,23.783,2001.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)
27342,629e7392924ea1d650df48d9,ULLAPOOL,UNKNOWN,SY2,NORTHERN LIGHTS,A24446,22265,NON-SECTOR,9.50,8.12,69.0,64.300,1978.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)
27343,629e7392924ea1d650df48da,ULLAPOOL,UNKNOWN,UL597,SOPHIE JANE,C17525,22724,NON-SECTOR,5.62,1.03,37.3,27.575,1989.0,F,GBR,Y,N,2019,CATEGORY A (10 METRE AND UNDER)


#### Checking the data to mongoDB - THIS IS AN UPDATE

In [40]:

# check if all items where inserted
print(len(uk_under10_df))
print(len(uk_over10_df))
print(uf_collection.count_documents({}))

21074
6270
27344


Although previous checks showed there were no NaN values in the entries prior to being added to the database, as an extra step, the keys in the collection are rechecked and replaced with "". This maintains the integrity that no NaN values are allowed in MongoDB.

In [41]:
# check for NaN values and replace it
for key in uf_collection.find_one({}).keys():
    uf_collection.update_many({key: float('nan')}, {'$unset': {key:''}})